# 22: REGRESIÓN LINEAL CON VARIABLES CATEGÓRICAS

Cuando tenemos variables categóricas que puedan influenciar a un modelo de regresión lineal, lo que hay que hacer es obtener variables dummies para cada una de las variables categóricas. De este modo, crearemos tantas variables dummies como valores puede tomar la variable categórica.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv('../../Recursos_curso/datasets/ecom-expense/Ecom Expense.csv')
data.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485


Tenemos 2 variables categóricas que pueden ser de interes: 'Gender' y 'City Tier'. Por otro lado, la variable a predecir será 'Total Spend'.

### Transformar las variables dummies

In [4]:
# Para la variable Gender

gender = pd.get_dummies(data['Gender'], prefix = 'Género')
gender.head()

,Género_Female,Género_Male
0,True,False
1,True,False
2,False,True
3,True,False
4,True,False


In [5]:
# Para la variable City Tier

city = pd.get_dummies(data['City Tier'], prefix = 'Ciudad')
city.head()

,Ciudad_Tier 1,Ciudad_Tier 2,Ciudad_Tier 3
0,True,False,False
1,False,True,False
2,False,True,False
3,True,False,False
4,False,True,False


### Juntar los dataframes dummies con el dataframe original

In [7]:
# Concatenamos los dataframes

dataframe = pd.concat([data, gender, city], axis = 1)
dataframe.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Género_Female,Género_Male,Ciudad_Tier 1,Ciudad_Tier 2,Ciudad_Tier 3
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,True,False,True,False,False
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,True,False,False,True,False
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,False,True,False,True,False
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,True,False,True,False,False
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,True,False,False,True,False


### Modelo de regresión lineal multiparametral

In [22]:
# Elaboración del modelo de regresión lineal

variables = dataframe.columns.values.tolist()
variables.remove('Total Spend') # quitamos la variable de respuesta
variables.remove('Transaction ID') # quitamos la variable que nos da problmeas ya que es identificadora
variables.remove('Gender') # quitamos la variable que nos da problmeas ya que es categórica
variables.remove('City Tier') # quitamos la variable que nos da problmeas ya que es categórica
regresion_mutiple = LinearRegression().fit(dataframe[variables], dataframe['Total Spend'])

In [23]:
regresion_mutiple.intercept_

-643.7731838037598

In [24]:
regresion_mutiple.coef_

array([ 6.18968657e+00,  3.93442110e+01,  1.47731711e-01,  1.69048699e-01,
        7.71442188e+02, -1.31627781e+02,  1.31627781e+02,  6.98809779e+01,
        5.04144467e+01, -1.20295425e+02])

In [26]:
list(zip(variables, regresion_mutiple.coef_))

[('Age ', 6.189686573746989),
 (' Items ', 39.344210969966134),
 ('Monthly Income', 0.14773171095344292),
 ('Transaction Time', 0.16904869866853314),
 ('Record', 771.4421878591896),
 ('Género_Female', -131.62778138807417),
 ('Género_Male', 131.62778138807397),
 ('Ciudad_Tier 1', 69.88097793487329),
 ('Ciudad_Tier 2', 50.414446652043075),
 ('Ciudad_Tier 3', -120.29542458691631)]

MODELO

Total Spend = - 643.7731838037598 + (6.189686573746989 * Age) + (39.344210969966134 * Items) + (0.14773171095344292 * Monthly Income) + (0.16904869866853314 * Transaction Time) + (771.4421878591896 * Record) + (- 131.62778138807417 * Género_Female) + (131.62778138807417 * Género_Male) + (69.88097793487329 * Ciudad_Tier 1) * (50.414446652043075 * Ciudad_Tier 2) + (- 120.29542458691631 * Ciudad_Tier 3)

Como podemos ver, este modelo tiene demasiados parametros, es muy posible que no todos sean relevantes

### Predicción del modelo - predict

In [38]:
# Predicción de los datos a partir del modelo

dataframe['Predicción'] = regresion_mutiple.predict(dataframe[variables])
dataframe.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Género_Female,Género_Male,Ciudad_Tier 1,Ciudad_Tier 2,Ciudad_Tier 3,Predicción
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,True,False,True,False,False,4991.568380
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,True,False,False,True,False,4695.893937
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,False,True,False,True,False,5327.444863
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,True,False,True,False,False,8241.840278
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,True,False,False,True,False,3469.255535


In [42]:
# Parametros del modelo

r2 = regresion_mutiple.score(dataframe[variables], dataframe['Total Spend'])
ssd = sum((dataframe['Total Spend'] - dataframe['Predicción']) ** 2)
rse = np.sqrt(ssd / (len(dataframe) - len(variables) - 1))
error = 100 * rse / np.mean(dataframe['Total Spend'])

print(f'El parametro R2 del modelo es: {r2}')
print(f'El parametro SSD del modelo es: {ssd}')
print(f'El parametro RSE del modelo es: {rse}')
print(f'El parametro Error del modelo es: {error}%')

El parametro R2 del modelo es: 0.922356824990408
El parametro SSD del modelo es: 1436959817.9987273
El parametro RSE del modelo es: 781.8005965026696
El parametro Error del modelo es: 12.685027066173522%


### Modelo eliminando dummy variables redundantes

Cuando se trabaja con dummy variables, no es necesario tener una columna por cada posible resultado de la categoria. Es decir, si tenemos 2 posibles respuestas para una variable categorica (Gener = Female o Male), basta con tener una única variable dummy para diferenciar entre las dos respuestas. Si por ejemplo tuvieramos 3 respuestas posibles, bastaría con tener solo 2 variables.

In [49]:
# Crear el dataframe con las dummy dataframes justas y necesarias

dataframe_2 = pd.concat([data, gender.iloc[:, 1:], city.iloc[:, 1:]], axis = 1) # con ilo[:,1:] quitamos la primera columna
dataframe_2.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Género_Male,Ciudad_Tier 2,Ciudad_Tier 3
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,False,False,False
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,False,True,False
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,True,True,False
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,False,False,False
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,False,True,False


In [50]:
# Elaborar el nuevo modelo

variables = dataframe_2.columns.values.tolist()
variables.remove('Total Spend') # quitamos la variable de respuesta
variables.remove('Transaction ID') # quitamos la variable que nos da problmeas ya que es identificadora
variables.remove('Gender') # quitamos la variable que nos da problmeas ya que es categórica
variables.remove('City Tier') # quitamos la variable que nos da problmeas ya que es categórica
regresion_mutiple = LinearRegression().fit(dataframe[variables], dataframe['Total Spend'])

In [53]:
regresion_mutiple.intercept_

-705.5199872567955

In [51]:
list(zip(variables, regresion_mutiple.coef_))

[('Age ', 6.189686573746995),
 (' Items ', 39.34421096996577),
 ('Monthly Income', 0.14773171095343313),
 ('Transaction Time', 0.16904869866852548),
 ('Record', 771.4421878591891),
 ('Género_Male', 263.25556277615027),
 ('Ciudad_Tier 2', -19.466531282829152),
 ('Ciudad_Tier 3', -190.1764025217907)]

Como se puede ver, lo únicoque ha variado es el valor de intercept y los coeficientes que están multiplicando a las dummy variables

In [60]:
# Predicción

dataframe_2['Predicción'] = regresion_mutiple.predict(dataframe_2[variables])
dataframe.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Género_Female,Género_Male,Ciudad_Tier 1,Ciudad_Tier 2,Ciudad_Tier 3,Predicción
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,True,False,True,False,False,4991.568380
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,True,False,False,True,False,4695.893937
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,False,True,False,True,False,5327.444863
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,True,False,True,False,False,8241.840278
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,True,False,False,True,False,3469.255535


In [61]:
# Parametros del modelo

r2 = regresion_mutiple.score(dataframe_2[variables], dataframe_2['Total Spend'])
ssd = sum((dataframe_2['Total Spend'] - dataframe_2['Predicción']) ** 2)
rse = np.sqrt(ssd / (len(dataframe_2) - len(variables) - 1))
error = 100 * rse / np.mean(dataframe_2['Total Spend'])

print(f'El parametro R2 del modelo es: {r2}')
print(f'El parametro SSD del modelo es: {ssd}')
print(f'El parametro RSE del modelo es: {rse}')
print(f'El parametro Error del modelo es: {error}%')

El parametro R2 del modelo es: 0.922356824990408
El parametro SSD del modelo es: 1436959817.998727
El parametro RSE del modelo es: 781.4682689227219
El parametro Error del modelo es: 12.679634918399103%
